In [ ]:
import os
import pandas as pd
import datetime
import string
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def month_string_to_number(string):    
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }
    s = string.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        print(s)
        raise ValueError('Not a month')

In [ ]:
def reviews_count_am(path_to_file: str, award_date):
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    count_before = 0
    count_after = 0
    
    for row in df.itertuples():
        # convert date on which the review was posted to datetime format
        if type(row.date) == str:
            date = row.date[row.date.find('on')+3:]
            review_year = int(date[-4:])
            review_month = month_string_to_number(date[:date.find(',')][:date.find(' ')])
            review_day = int(date[:date.find(',')][date.find(' ')+1:])
            review_date = datetime.date(review_year, review_month, review_day)

            if review_date < award_date:
                count_before += 1
            else:
                count_after += 1
            
    return count_before, count_after

In [ ]:
def reviews_count_gr(path_to_file: str, award_date):
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    count_before = 0
    count_after = 0
    
    for row in df.itertuples():
        # convert date on which the review was posted to datetime format
        review_year = int(row.date[0:4])
        review_month = int(row.date[5:7])
        review_day = int(row.date[8:10])
        review_date = datetime.date(review_year, review_month, review_day)
        
        if review_date < award_date:
            count_before += 1
        else:
            count_after += 1
            
    return count_before, count_after

In [ ]:
def reviews_rating_am(path_to_file: str, award_date):
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    before_rating_list = []
    after_rating_list = []

    for row in df.itertuples():
        # convert date on which the review was posted to datetime format
        if type(row.date) == str:
            date = row.date[row.date.find('on')+3:]
            review_year = int(date[-4:])
            review_month = month_string_to_number(date[:date.find(',')][:date.find(' ')])
            review_day = int(date[:date.find(',')][date.find(' ')+1:])
            review_date = datetime.date(review_year, review_month, review_day)

            review_rate = 0

            if row.rating == '5.0 out of 5 stars':
                review_rate = 5
            elif row.rating == '4.0 out of 5 stars':
                review_rate = 4
            elif row.rating == '3.0 out of 5 stars':
                review_rate = 3
            elif row.rating == '2.0 out of 5 stars':
                review_rate = 2
            elif row.rating == '1.0 out of 5 stars':
                review_rate = 1

            if review_date < award_date:
                before_rating_list.append(review_rate)
            else:
                after_rating_list.append(review_rate)

    return before_rating_list, after_rating_list

In [ ]:
def reviews_rating_am_each_book(path_to_file: str, award_date):    
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    sum_rate_before = 0
    sum_rate_after = 0
    count_before = 0
    count_after = 0
    
    for row in df.itertuples():
        # convert date on which the review was posted to datetime format
        if type(row.date) == str:
            date = row.date[row.date.find('on')+3:]
            review_year = int(date[-4:])
            review_month = month_string_to_number(date[:date.find(',')][:date.find(' ')])
            review_day = int(date[:date.find(',')][date.find(' ')+1:])
            review_date = datetime.date(review_year, review_month, review_day)

            review_rate = 0

            if row.rating == '5.0 out of 5 stars':
                review_rate = 5
            elif row.rating == '4.0 out of 5 stars':
                review_rate = 4
            elif row.rating == '3.0 out of 5 stars':
                review_rate = 3
            elif row.rating == '2.0 out of 5 stars':
                review_rate = 2
            elif row.rating == '1.0 out of 5 stars':
                review_rate = 1

            if review_date < award_date:
                count_before += 1
                sum_rate_before += review_rate
            else:
                count_after += 1
                sum_rate_after += review_rate
                
    if count_before != 0 and count_after != 0:
        return sum_rate_before/count_before, sum_rate_after/count_after
    else:
        return -1, -1

In [ ]:
def reviews_rating_gr(path_to_file: str, award_date):
    # did not like it : 1 star
    # it was ok : 2 stars
    # liked it : 3 stars
    # really liked it: 4 stars
    # it was amazing: 5 stars
    # not rated: n/a
    
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    before_rating_list = []
    after_rating_list = []

    for row in df.itertuples():
        # convert date on which the review was posted to datetime format
        review_year = int(row.date[0:4])
        review_month = int(row.date[5:7])
        review_day = int(row.date[8:10])
        review_date = datetime.date(review_year, review_month, review_day)
        
        review_rate = 0
    
        if row.rating == 'it was amazing':
            review_rate = 5
        elif row.rating == 'really liked it':
            review_rate = 4
        elif row.rating == 'liked it':
            review_rate = 3
        elif row.rating == 'it was ok':
            review_rate = 2
        elif row.rating == 'did not like it':
            review_rate = 1

        if review_date < award_date:
            before_rating_list.append(review_rate)
        else:
            after_rating_list.append(review_rate)

    return before_rating_list, after_rating_list

In [ ]:
def reviews_rating_gr_each_book(path_to_file: str, award_date):
    # did not like it : 1 star
    # it was ok : 2 stars
    # liked it : 3 stars
    # really liked it: 4 stars
    # it was amazing: 5 stars
    # not rated: n/a
    
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    sum_rate_before = 0
    sum_rate_after = 0
    count_before = 0
    count_after = 0
    
    for row in df.itertuples():
        # convert date on which the review was posted to datetime format
        review_year = int(row.date[0:4])
        review_month = int(row.date[5:7])
        review_day = int(row.date[8:10])
        review_date = datetime.date(review_year, review_month, review_day)
        
        review_rate = 0
    
        if row.rating == 'it was amazing':
            review_rate = 5
        elif row.rating == 'really liked it':
            review_rate = 4
        elif row.rating == 'liked it':
            review_rate = 3
        elif row.rating == 'it was ok':
            review_rate = 2
        elif row.rating == 'did not like it':
            review_rate = 1
    
        if review_date < award_date:
            count_before += 1
            sum_rate_before += review_rate
        else:
            count_after += 1
            sum_rate_after += review_rate
            
    return sum_rate_before/count_before, sum_rate_after/count_after

In [ ]:
def reviews_count(path_to_file: str, award_date):
    if path_to_file[58:60] == 'am':
        return reviews_count_am(path_to_file, award_date)
    elif path_to_file[58:60] == 'gr':
        return reviews_count_gr(path_to_file, award_date)
    
def reviews_rating(path_to_file: str, award_date):
    if path_to_file[58:60] == 'am':
        return reviews_rating_am(path_to_file, award_date)
    elif path_to_file[58:60] == 'gr':
        return reviews_rating_gr(path_to_file, award_date)

In [ ]:
def reviews_to_tfidf(path_to_file: str, award_date):    
    # read in reviews for each book
    df = pd.read_csv(path_to_file, encoding = 'ISO-8859-1')
    
    before_reviews = []
    after_reviews = []
    
    # process each review
    for row in df.itertuples():
        
        # lowercase, strip punctuation, and convert it to a list of words
        text = str(row.review)
        text_no_pun = text.translate(str.maketrans('', '', string.punctuation))
        #word_list = text_no_pun.lower().split()
        
        # convert date on which the review was posted to datetime format
        review_year = int(row.date[0:4])
        review_month = int(row.date[5:7])
        review_day = int(row.date[8:10])
        review_date = datetime.date(review_year, review_month, review_day)
        
        if review_date < award_date:
            before_reviews.append(text_no_pun)

        else:
            after_reviews.append(text_no_pun)
            
    vectorizer = TfidfVectorizer()
    
    tfidf_before_reviews = vectorizer.fit_transform(before_reviews)
    
    # place tf-idf values in a pandas data frame
    before_pd = pd.DataFrame(tfidf_before_reviews.T.todense(), index=vectorizer.get_feature_names())
    
    tfidf_after_reviews = vectorizer.fit_transform(after_reviews)
    
    # place tf-idf values in a pandas data frame
    after_pd = pd.DataFrame(tfidf_after_reviews.T.todense(), index=vectorizer.get_feature_names())
 
    #df.sort_values(by=["tfidf"],ascending=False)
 
    return before_pd, after_pd
    
    #words_before_filt = {k : v for k, v in words_before.items() if v > 5}
    #words_after_filt = {k : v for k, v in words_after.items() if v > 5}
    
    #for k, v in words_before.items():
    #    if v < 5:
    #        del words_before[k]
    
    #for k, v in words_after.items():
    #    if v < 5:
     #       del words_after[k]
    
    #words_before_norm = {k : v / before_review_count for k, v in words_before.items()}
    #words_after_norm = {k : v / after_review_count for k, v in words_after.items()}
    
    #return words_before_norm, words_after_norm

In [ ]:
# Booker prize winner announcement dates
eng_date_2019 = datetime.date(2019, 10, 14)
eng_date_2018 = datetime.date(2018, 10, 16)
eng_date_2017 = datetime.date(2017, 10, 17)
eng_date_2016 = datetime.date(2016, 10, 25)

# Booker international prize winner announcement dates
int_date_2019 = datetime.date(2019, 5, 21)
int_date_2018 = datetime.date(2018, 5, 22)
int_date_2017 = datetime.date(2017, 6, 14)
int_date_2016 = datetime.date(2016, 5, 16)

In [ ]:
path_to_folder = os.getcwd() + '\\data\\booker_reviews_am'
#path_to_folder = os.getcwd() + '\\data\\booker_reviews_amuk'
#path_to_folder = os.getcwd() + '\\data\\booker_reviews_gr'

In [ ]:
int_winner_review_count = []
int_winner_review_rate = []
int_nominee_review_count = []
int_nominee_review_rate = []
eng_winner_review_count = []
eng_winner_review_rate = []
eng_nominee_review_count = []
eng_nominee_review_rate = []

# process each year
for folder_name in os.listdir(path_to_folder):
    date = datetime.date(1900, 1, 1)
    
    if folder_name[3:] == 'eng_2016':
        date = eng_date_2016
    elif folder_name[3:] == 'eng_2017':
        date = eng_date_2017
    elif folder_name[3:] == 'eng_2018':
        date = eng_date_2018
    elif folder_name[3:] == 'eng_2019':
        date = eng_date_2019
    elif folder_name[3:] == 'int_2016':
        date = int_date_2016
    elif folder_name[3:] == 'int_2017':
        date = int_date_2017
    elif folder_name[3:] == 'int_2018':
        date = int_date_2018
    elif folder_name[3:] == 'int_2019':
        date = int_date_2019
    
    # process each file
    for file_name in os.listdir(path_to_folder + '\\' + folder_name):
        # process .csv files
        if file_name[-3:] == 'csv':
            # international
            if folder_name[3:6] == 'int':
                 # winner (file name starts with '_')
                if file_name[0] == '_':
                    int_winner_review_count.append(reviews_count(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
                    int_winner_review_rate.append(reviews_rating(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
                # shortlisted
                else:
                    int_nominee_review_count.append(reviews_count(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
                    int_nominee_review_rate.append(reviews_rating(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
            elif folder_name[3:6] == 'eng':
                # winner
                if file_name[0] == '_':
                    eng_winner_review_count.append(reviews_count(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
                    eng_winner_review_rate.append(reviews_rating(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
                # shortlisted
                else:
                    eng_nominee_review_count.append(reviews_count(path_to_folder + '\\' + folder_name + '\\' + file_name, date))
                    eng_nominee_review_rate.append(reviews_rating(path_to_folder + '\\' + folder_name + '\\' + file_name, date))

In [ ]:
all_ratings = []

for book in eng_winner_review_rate:
    for rating in book[0]:
        all_ratings.append(['winner_before', 'eng', rating])
    for rating in book[1]:
        all_ratings.append(['winner_after', 'eng', rating])
        
for book in eng_nominee_review_rate:
    for rating in book[0]:
        all_ratings.append(['nominee_before', 'eng', rating])
    for rating in book[1]:
        all_ratings.append(['nominee_after', 'eng', rating])
        
for book in int_winner_review_rate:
    for rating in book[0]:
        all_ratings.append(['winner_before', 'int', rating])
    for rating in book[1]:
        all_ratings.append(['winner_after', 'int', rating])
        
for book in int_nominee_review_rate:
    for rating in book[0]:
        all_ratings.append(['nominee_before', 'int', rating])
    for rating in book[1]:
        all_ratings.append(['nominee_after', 'int', rating])

In [ ]:
df_all_rate = pd.DataFrame.from_records(all_ratings, columns=["status", "eng_or_int", "review_rating"])
path = 'C:\\Users\\Jungeun (June) Lim\\Documents\\code\\all_ratings_am_to2018.csv'
df_all_rate.to_csv(path, index = False)

In [ ]:
"""
final_list = []

for i, j in zip(int_winner_review_count, int_winner_review_rate):
    final_list.append(['winner_before', 'int', i[0], j[0]])
    final_list.append(['winner_after', 'int', i[1], j[1]])
    
for i, j in zip(int_nominee_review_count, int_nominee_review_rate):
    final_list.append(['nominee_before', 'int', i[0], j[0]])
    final_list.append(['nominee_after', 'int', i[1], j[1]])
    
for i, j in zip(eng_winner_review_count, eng_winner_review_rate):
    final_list.append(['winner_before', 'eng', i[0], j[0]])
    final_list.append(['winner_after', 'eng', i[1], j[1]])
    
for i, j in zip(eng_nominee_review_count, eng_nominee_review_rate):
    final_list.append(['nominee_before', 'eng', i[0], j[0]])
    final_list.append(['nominee_after', 'eng', i[1], j[1]])
"""

In [ ]:
df_final = pd.DataFrame.from_records(final_list, columns=["status", "eng_or_int", "review_count", "review_rating"])
df_final.to_csv(path_to_folder + '\\final_data_am.csv', index = False)

In [ ]:
winner_after_sum = 0
winner_before_sum = 0

for t in int_winner_review_count:
    winner_before_sum += t[0]
    winner_after_sum += t[1]
    
print(winner_before_sum/len(int_winner_review_count))
print(winner_after_sum/len(int_winner_review_count))

In [ ]:
winner_after_sum = 0
winner_before_sum = 0

for t in eng_winner_review_count:
    winner_before_sum += t[0]
    winner_after_sum += t[1]
    
print(winner_before_sum/len(eng_winner_review_count))
print(winner_after_sum/len(eng_winner_review_count))

In [ ]:
nominee_after_sum = 0
nominee_before_sum = 0

for t in int_nominee_review_count:
    nominee_before_sum += t[0]
    nominee_after_sum += t[1]
    
print(nominee_before_sum/len(int_nominee_review_count))
print(nominee_after_sum/len(int_nominee_review_count))

In [ ]:
nominee_after_sum = 0
nominee_before_sum = 0

for t in eng_nominee_review_count:
    nominee_before_sum += t[0]
    nominee_after_sum += t[1]
    
print(nominee_before_sum/len(eng_nominee_review_count))
print(nominee_after_sum/len(eng_nominee_review_count))

In [ ]:
winner_after_sum = 0
winner_before_sum = 0

for t in int_winner_review_rate:
    winner_before_sum += t[0]
    winner_after_sum += t[1]
    
print(winner_before_sum/len(int_winner_review_rate))
print(winner_after_sum/len(int_winner_review_rate))

In [ ]:
nominee_after_sum = 0
nominee_before_sum = 0

for t in int_nominee_review_rate:
    nominee_before_sum += t[0]
    nominee_after_sum += t[1]
    
print(nominee_before_sum/len(int_nominee_review_rate))
print(nominee_after_sum/len(int_nominee_review_rate))

In [ ]:
winner_after_sum = 0
winner_before_sum = 0

for t in eng_winner_review_rate:
    winner_before_sum += t[0]
    winner_after_sum += t[1]
    
print(winner_before_sum/len(eng_winner_review_rate))
print(winner_after_sum/len(eng_winner_review_rate))

In [ ]:
winner_after_sum = 0
winner_before_sum = 0

for t in eng_nominee_review_rate:
    winner_before_sum += t[0]
    winner_after_sum += t[1]
    
print(winner_before_sum/len(eng_nominee_review_rate))
print(winner_after_sum/len(eng_nominee_review_rate))

In [ ]:
for folder_name in os.listdir(path_to_folder):
    
    date = datetime.date(1900, 1, 1)
    
    if folder_name == 'gr_eng_2016':
        date = eng_date_2016
    elif folder_name == 'gr_eng_2017':
        date = eng_date_2017
    elif folder_name == 'gr_eng_2018':
        date = eng_date_2018
    elif folder_name == 'gr_eng_2019':
        date = eng_date_2019
    elif folder_name == 'gr_int_2016':
        date = int_date_2016
    elif folder_name == 'gr_int_2017':
        date = int_date_2017
    elif folder_name == 'gr_int_2018':
        date = int_date_2018
    elif folder_name == 'gr_int_2019':
        date = int_date_2019
        
    for file_name in os.listdir(path_to_folder + '\\' + folder_name):
        if file_name[-3:] == 'csv':
            tfidf = reviews_to_tfidf(path_to_folder + '\\' + folder_name + '\\' + file_name, date)

            with open(path_to_folder + '\\' + folder_name + '\\' + file_name[:-4] + '.pkl', 'wb') as f:
                pickle.dump(tfidf, f)

Sanity check:

In [ ]:
file_name[0]

In [ ]:
df2 = pd.read_pickle(path_to_folder + '\\' + folder_name + '\\' + file_name[:-4] + '.pkl')

In [ ]:
df2[1][48]['author']

In [ ]:
df2[1]['author']

In [ ]:
df2[0]['of']

In [ ]:
df2[1]['of']